Query1 : Find the atoms of a left hand molecule whose sum of the distances to a Polymer is less than X.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
!pip install biopandas
from biopandas.mol2 import PandasMol2
import pandas as pd
import numpy as np

     |████████████████████████████████| 264 kB 4.1 MB/s 


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Load molecular data
pmol = PandasMol2().read_mol2('/content/drive/My Drive/Molecular Simulation Data/flavanone/Copy of traj1_file.mol2')


# Display data
from IPython.display import display, HTML
#print ("All atoms in the data:")
display(HTML(pmol.df.to_html()))

#print('Molecule ID: %s' % pmol.code)
#print('\nRaw MOL2 file contents:\n\n%s\n...' % pmol.mol2_text[:1000])
#pmol.df.head(10)

,atom_id,atom_name,x,y,z,atom_type,subst_id,subst_name,charge
0,1,C1,8.5526,-3.7802,24.8711,cb,1,CSP,-0.2524
1,2,C2,7.6277,-2.7060,24.1949,cb,1,CSP,0.0178
2,3,C3,8.4331,-1.5272,23.6360,cb,1,CSP,-0.0128
3,4,C4,9.5162,-0.9819,24.6243,cb,1,CSP,0.0864
4,5,O5,10.2702,-2.0725,25.1265,ob,1,CSP,-0.2548
5,6,C6,9.5359,-3.0798,25.8033,cb,1,CSP,0.0352
6,7,C7,10.6146,-4.0377,26.3204,c3,1,CSP,-0.0122
7,8,O8,10.8475,-3.8405,27.7522,os,1,CSP,-0.4082
8,9,C9,11.6793,-2.7429,28.0752,c,1,CSP,0.7524
9,10,O10,12.4410,-2.1327,27.3308,o,1,CSP,-0.5146


In [22]:
# Delete and Rename columns
# Replace RFL and SFL to R-drug and S-drug respectively
required_hbond_atom = pmol.df.drop(
    columns=['atom_id', 'x', 'y', 'z', 'subst_name', 'charge']).rename(
        columns={"atom_name":"Atoms","atom_type":"Atom Type","subst_id": "Mol no"}).replace(
            to_replace=['RFL','SFL'], value=['R-drug','S-drug'], regex=True)
        

In [23]:
donor = (pmol.df['atom_type'] =='n') & (pmol.df['subst_name'] =='CSP')
acceptor = ((pmol.df['atom_type'] =='o')|(pmol.df['atom_type'] =='os')) & ((pmol.df['subst_name'] =='RFL')|(pmol.df['subst_name'] =='SFL'))

#required_hbond_atom['Donor'],required_hbond_atom['Acceptor']= [donor*1, acceptor*1]
#required_hbond_atom.loc[437:455,:]
#required_hbond_atom.style.hide_index().set_properties(**{'text-align': 'center'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

In [24]:
# Calculate the atoms coordinates
#all_atoms_coord = pmol.df.loc[:,['x', 'y', 'z']].values

#from scipy.spatial import distance
# Calculate the distances between all atoms
#atoms_dist = distance.cdist(all_atoms_coord, all_atoms_coord, 'euclidean')
#atoms_dist  = np.round(atoms_dist, 2)


In [25]:
# Calculate the x coordinates
atoms_x_coord = pmol.df.loc[:,'x'].values

x_coord_diff = abs(atoms_x_coord[np.newaxis,:] - atoms_x_coord[:,np.newaxis])

#x_coord_diff = x_coord_diff.flatten()

#x_coord_diff = [abs(item - 72) if item>36 else item for item in x_coord_diff]

#for index, item in enumerate(x_coord_diff):
	#if item > 36:
		#x_coord_diff[index] = abs(x_coord_diff[index] - 72)

for i in range(len(x_coord_diff)):
  for j in range(len(x_coord_diff[i])):
    if x_coord_diff[i][j] > 36:
      x_coord_diff[i][j] = abs(x_coord_diff[i][j] - 72)


In [26]:
# Calculate the y coordinates
atoms_y_coord = pmol.df.loc[:,'y'].values

y_coord_diff = abs(atoms_y_coord[np.newaxis,:] - atoms_y_coord[:,np.newaxis])

for i in range(len(y_coord_diff)):
  for j in range(len(y_coord_diff[i])):
    if y_coord_diff[i][j] > 36:
      y_coord_diff[i][j] = abs(y_coord_diff[i][j] - 72)


In [27]:
# Calculate the y coordinates
atoms_z_coord = pmol.df.loc[:,'z'].values
z_coord_diff = abs(atoms_z_coord[np.newaxis,:] - atoms_z_coord[:,np.newaxis])

atoms_dist = np.sqrt(np.square(x_coord_diff) + np.square(y_coord_diff) + np.square(z_coord_diff))
atoms_dist = np.round(atoms_dist,2)


In [28]:
#import sys
#np.set_printoptions(threshold=sys.maxsize)

all_atoms_dist = pd.DataFrame(data = atoms_dist, index =  pmol.df.index , columns = pmol.df.index )
#all_atoms_dist = pd.DataFrame(data = atoms_dist, index=  pmol.df.index , columns = pmol.df['atom_name'].values)
#HTML(all_atoms_dist.to_html())


In [29]:
hbond_atom_distance = pd.concat([required_hbond_atom, all_atoms_dist] , axis=1)
#hbond_atom_distance.style.hide_index().set_properties(**{'text-align': 'center'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
#display(HTML(hbond_atom_distance.to_html()))

In [32]:
# Define each drug and polymer molecules

R_drug_5 = list(range(5836,5865))
R_drug_6 = list(range(5865,5894))
R_drug_7 = list(range(5894,5923))
R_drug_8 = list(range(5923,5952))
R_drug_9 = list(range(5952,5981))
S_drug_10 = list(range(5981,6010))
S_drug_11 = list(range(6010,6039))
S_drug_12 = list(range(6039,6068))
S_drug_13 = list(range(6068,6097))
S_drug_14 = list(range(6097,6126))

CSP_1 = list(range(0,1459))
CSP_2 = list(range(1459,2918))
CSP_3 = list(range(2918,4377))
CSP_4 = list(range(4377,5836))
CSP = range(0,5836)

In [47]:
selected_drug_mol = S_drug_10 
polymer_mol = CSP_1

min_dist_1 = np.min(atoms_dist[selected_drug_mol[0]:selected_drug_mol[-1]+1, polymer_mol], axis =1)

min_dist_CSP1 = np.argmin(atoms_dist[selected_drug_mol[0]:selected_drug_mol[-1]+1, polymer_mol], axis =1)

min_dist_CSP1_atom = pmol.df.loc[min_dist_CSP1,'atom_name'].values
min_dist_CSP1_atom_type = pmol.df.loc[min_dist_CSP1,'atom_type'].values

polymer_mol = CSP_2

min_dist_2 = np.min(atoms_dist[selected_drug_mol[0]:selected_drug_mol[-1]+1, polymer_mol], axis =1)

min_dist_CSP2 = np.argmin(atoms_dist[selected_drug_mol[0]:selected_drug_mol[-1]+1, polymer_mol], axis =1)

min_dist_CSP2_atom = pmol.df.loc[min_dist_CSP2,'atom_name'].values
min_dist_CSP2_atom_type = pmol.df.loc[min_dist_CSP2,'atom_type'].values

polymer_mol = CSP_3

min_dist_3 = np.min(atoms_dist[selected_drug_mol[0]:selected_drug_mol[-1]+1, polymer_mol], axis =1)

min_dist_CSP3 = np.argmin(atoms_dist[selected_drug_mol[0]:selected_drug_mol[-1]+1, polymer_mol], axis =1)

min_dist_CSP3_atom = pmol.df.loc[min_dist_CSP3,'atom_name'].values
min_dist_CSP3_atom_type = pmol.df.loc[min_dist_CSP3,'atom_type'].values

polymer_mol = CSP_4

min_dist_4 = np.min(atoms_dist[selected_drug_mol[0]:selected_drug_mol[-1]+1, polymer_mol], axis =1)

min_dist_CSP4 = np.argmin(atoms_dist[selected_drug_mol[0]:selected_drug_mol[-1]+1, polymer_mol], axis =1)

min_dist_CSP4_atom = pmol.df.loc[min_dist_CSP4,'atom_name'].values
min_dist_CSP4_atom_type = pmol.df.loc[min_dist_CSP4,'atom_type'].values


In [51]:
drug_atoms_distances = required_hbond_atom.loc[selected_drug_mol]
drug_atoms_distances['Min Dist from CSP1'], drug_atoms_distances['CSP1 Atom'], drug_atoms_distances['CSP1 Atom Type'] = [min_dist_1, min_dist_CSP1_atom, min_dist_CSP1_atom_type]
drug_atoms_distances['Min Dist from CSP2'], drug_atoms_distances['CSP2 Atom'], drug_atoms_distances['CSP2 Atom Type'] = [min_dist_2, min_dist_CSP2_atom, min_dist_CSP2_atom_type]
drug_atoms_distances['Min Dist from CSP3'], drug_atoms_distances['CSP3 Atom'], drug_atoms_distances['CSP3 Atom Type'] = [min_dist_3, min_dist_CSP3_atom, min_dist_CSP3_atom_type]
drug_atoms_distances['Min Dist from CSP4'], drug_atoms_distances['CSP4 Atom'], drug_atoms_distances['CSP4 Atom Type'] = [min_dist_4, min_dist_CSP4_atom, min_dist_CSP4_atom_type]
print("         Distances of Falavanone S Drug Molecule 1 to Polymers (CSP1-CSP4):")
drug_atoms_distances

         Distances of Falavanone S Drug Molecule 1 to Polymers (CSP1-CSP4):


,Atoms,Atom Type,Mol no,Min Dist from CSP1,CSP1 Atom,CSP1 Atom Type,Min Dist from CSP2,CSP2 Atom,CSP2 Atom Type,Min Dist from CSP3,CSP3 Atom,CSP3 Atom Type,Min Dist from CSP4,CSP4 Atom,CSP4 Atom Type
5981,O1,os,10,3.06,H418,hc,7.89,H381,hc,26.91,H517,hc,11.17,H438,hc
5982,O2,o,10,6.27,H418,hc,7.07,H386,hc,25.41,H517,hc,14.91,H439,hc
5983,C1,c3,10,3.35,H418,hc,7.63,H322,hc,26.61,H517,hc,11.72,H439,hc
5984,C2,c3,10,4.73,H418,hc,6.81,H322,hc,25.55,H517,hc,13.16,H439,hc
5985,C3,ca,10,3.55,H418,hc,7.06,H381,hc,26.43,H517,hc,11.20,H439,hc
5986,C4,c,10,5.19,H418,hc,7.43,H386,hc,25.82,H517,hc,13.75,H439,hc
5987,C5,ca,10,4.61,H418,hc,8.27,H386,hc,26.88,H517,hc,12.96,H438,hc
5988,C6,ca,10,3.57,H418,hc,8.37,H475,hc,27.27,H517,hc,11.64,H438,hc
5989,C7,ca,10,3.86,H476,hc,5.78,H381,hc,25.32,H517,hc,11.67,H439,hc
5990,C8,ca,10,2.81,H419,hc,7.68,H321,hc,27.46,H517,hc,10.49,H439,hc


In [16]:
from statistics import mean
print ("Avg of Minimum Distances from CSP1 = ", mean(drug_atoms_distances['Minimum Distances from CSP1']))
print ("Avg of Minimum Distances from CSP2 = ", mean(drug_atoms_distances['Minimum Distances from CSP2']))
print ("Avg of Minimum Distances from CSP3 = ", mean(drug_atoms_distances['Minimum Distances from CSP3']))
print ("Avg of Minimum Distances from CSP4 = ", mean(drug_atoms_distances['Minimum Distances from CSP4']))

Avg of Minimum Distances from CSP1 =  28.775517241379312
Avg of Minimum Distances from CSP2 =  30.10896551724138
Avg of Minimum Distances from CSP3 =  22.673448275862068
Avg of Minimum Distances from CSP4 =  20.89034482758621


In [17]:
from statistics import mean
#cutoff_distance = mean(hbond_atom_distance.loc[CSP_1,Sdrug_14].sum())
#print("Value of X:", np.round(cutoff_distance,2),"A")

Query 1 : Find the atoms of a left hand molecule whose sum of the distances to a Polymer is less than X. 

---



Query 2 : Find the minimum distances of atoms of the drug molecules from each polymer substrate.

In [18]:
cutoff_distance = mean(hbond_atom_distance.loc[CSP_4,selected_drug_mol].sum())
attached_atoms_CSP4=[]

for drug_index in selected_drug_mol :
  distance_sum = hbond_atom_distance.loc[CSP_4,drug_index].sum()
  
  if distance_sum < cutoff_distance:
    attached_atoms_CSP4.append(drug_index)


print ("        Atoms of Falvanone S-drug mol no 5\n        whose sum of the distances to\n        Polymer 4 is less than value X:")
#required_hbond_atom["Atoms"].loc[attached_atoms_CSP4]

        Atoms of Falvanone S-drug mol no 5
        whose sum of the distances to
        Polymer 4 is less than value X:
